In [ ]:
import concurrent.futures
import traceback
from databricks.sdk.runtime import dbutils

base_path = "/Volumes/workspace/default/files"

# set catalog/schema from widgets (cells above set CATALOG/SCHEMA if needed)
spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA tpch100_db")

def load_table(t, path=None):
    if path is None:
        path = base_path+f"/{t}"
    try:
        sql = f"""COPY INTO {t}
                FROM '{path}'
                FILEFORMAT = PARQUET
            """
        spark.sql(sql)
        return (t, "OK")
    except Exception as e:
        # capture error and stack for debugging
        return (t, "ERROR", str(e), traceback.format_exc())

# tune max_workers as appropriate for your cluster/IO; avoid oversubscription
max_workers = 8

results = []
tables = [
    "nation", "region", "part", "supplier",
    "partsupp", "customer", "orders"
]
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as ex:
    futures = {ex.submit(load_table, t): t for t in tables}
    for fut in concurrent.futures.as_completed(futures):
        results.append(fut.result())

for r in results:
    print(r)

results = []
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as ex:
    t="lineitem"
    futures =[ex.submit(load_table, t, part.path) for part in dbutils.fs.ls(base_path+f"/{t}" )]
    for fut in concurrent.futures.as_completed(futures):
        results.append(fut.result())

# show summary
for r in results:
    print(r)